In [1]:
!pip3 install xgboost
!pip install decision-tree-morfist

In [5]:
import pandas as pd
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import statistics
from utils.pandas_dataframe import grid_display
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from itertools import permutations
import gc

# validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from morfist import MixedRandomForest

# grid search
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.model_selection import GridSearchCV


from sklearn.multioutput import RegressorChain
from sklearn.multioutput import MultiOutputRegressor

from IPython.display import display


%matplotlib inline

In [6]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format',lambda x: '%.2f' % x)
# pd.set_option('display.max_rows', 5)
pd.set_option('display.max_rows', None)

In [7]:
data = pd.read_excel('../Data/2022-train-v2.xlsx')

In [8]:
print(f"train shape : {data.shape}")

train shape : (635, 131)


In [9]:
data_y = data[['sensor_point5_i_value', 'sensor_point6_i_value',\
               'sensor_point7_i_value', 'sensor_point8_i_value',\
               'sensor_point9_i_value', 'sensor_point10_i_value']]
data_x = data.drop(['sensor_point5_i_value', 'sensor_point6_i_value',\
                    'sensor_point7_i_value', 'sensor_point8_i_value',\
                    'sensor_point9_i_value', 'sensor_point10_i_value'], axis=1)

In [10]:
display(data_x)

,clean_temp,clean_ec,clean_ph4,clean_ph5,clean_ph7,clean_ph8,clean_pressure11,clean_pressure12,clean_pressure21,clean_pressure22,...,env_rpi14_pm1,env_rpi14_pm10,env_rpi14_pm25,env_rpi14_temp,env_rpi15_hum,env_rpi15_lux,env_rpi15_pm1,env_rpi15_pm10,env_rpi15_pm25,env_rpi15_temp
0,41.10,12.40,NaN,NaN,NaN,6.90,820.24,1262.82,883.46,630.74,...,26.33,47.50,37.33,34.41,43.73,0.50,29.50,51.50,42.00,34.78
1,41.10,12.40,NaN,NaN,NaN,6.90,820.15,1263.00,883.60,630.70,...,26.33,47.50,37.33,34.41,43.73,0.50,29.50,51.50,42.00,34.78
2,41.10,12.40,NaN,NaN,NaN,6.90,820.15,1263.00,883.60,630.70,...,26.33,47.50,37.33,34.41,43.73,0.50,29.50,51.50,42.00,34.78
3,41.10,12.40,NaN,NaN,NaN,6.90,820.15,1263.00,883.60,630.70,...,26.33,47.50,37.33,34.41,43.73,0.50,29.50,51.50,42.00,34.78
4,41.10,12.40,NaN,NaN,NaN,6.90,820.78,1264.00,883.31,630.74,...,26.71,48.85,38.14,34.48,43.94,0.42,29.71,51.85,42.42,34.82
5,41.10,12.40,NaN,NaN,NaN,6.90,820.78,1264.00,883.31,630.74,...,26.71,48.85,38.14,34.48,43.94,0.42,29.71,51.85,42.42,34.82
6,41.10,12.40,NaN,NaN,NaN,6.90,820.78,1264.00,883.31,630.74,...,26.71,48.85,38.14,34.48,43.94,0.42,29.71,51.85,42.42,34.82
7,41.10,12.40,NaN,NaN,NaN,6.90,820.78,1264.00,883.31,630.74,...,26.71,48.85,38.14,34.48,43.94,0.42,29.71,51.85,42.42,34.82
8,41.90,12.50,NaN,NaN,NaN,7.00,821.01,1266.62,883.79,630.74,...,26.71,48.85,38.14,34.48,43.94,0.42,29.71,51.85,42.42,34.82
9,41.90,12.50,NaN,NaN,NaN,7.00,821.01,1266.62,883.79,630.74,...,26.71,48.85,38.14,34.48,43.94,0.42,29.71,51.85,42.42,34.82


In [11]:
def Col_types(Data):
    Column_types = Data.dtypes.to_frame().reset_index()
    Column_types.columns = ['ColumnName', 'Type']
    Column_types.sort_values( by= 'Type', inplace = True)
    return Column_types

def Missing_Counts(Data):
    missing = Data.isnull().sum()
    missing = missing[ missing >0]
    missing.sort_values( inplace=True)
    Missing_Count = pd.DataFrame({ 'ColumnName':missing.index, 'MissingCount':missing.values})
    Missing_Count['Percentage(%)'] = Missing_Count['MissingCount'].apply(lambda x:round(x/Data.shape[0]*100,2))
    return Missing_Count

### 檢查缺失值

In [12]:
display(Missing_Counts(data_x))

,ColumnName,MissingCount,Percentage(%)
0,clean_ph4,286,45.04
1,clean_ph5,286,45.04
2,clean_ph7,286,45.04


In [13]:
display(Missing_Counts(data_y))

,ColumnName,MissingCount,Percentage(%)


In [14]:
data_x = data_x.fillna(0)

### 檢查資料屬性

In [15]:
display(Col_types(data_x))

,ColumnName,Type
0,clean_temp,float64
90,painting_g11_act_hvv,float64
89,painting_g11_act_t_air,float64
88,painting_g11_act_f_air,float64
87,painting_g11_act_a_air,float64
86,painting_g10_act_hvc,float64
85,painting_g10_act_hvv,float64
84,painting_g10_act_t_air,float64
83,painting_g10_act_f_air,float64
82,painting_g10_act_a_air,float64


# 1.拆分訓練及測試集

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(data_x, data_y, train_size = 0.2, random_state = 3)

# X變數標準化
scaler = StandardScaler().fit(data_x)
data_x_S = scaler.transform(data_x)

# 2.迴圈計算每個Y在每個模型下的最佳超參數，並計算RMSE值

In [21]:
# 分別計算每個Model的最佳超參數及RMSE，以判斷6個 Y的最佳 Model
Lasso_model_list = []
Ridge_model_list = []
XGB_model_list = []
Lasso_train_RMSE = []
Ridge_train_RMSE = []
XGB_train_RMSE = []
Lasso_test_RMSE = []
Ridge_test_RMSE = []
XGB_test_RMSE = []
Model_result_train = pd.DataFrame({})
Model_result_test = pd.DataFrame({})

for i in range(6):
# 1. Lasso_model
    # Lasso超參數
    Lassocv =  LassoCV(alphas = [0.01,0.1,0.5,1,5,7,10,30,100,500], cv = 3, max_iter = 10000)
    Ridgecv = RidgeCV(alphas = [0.01,0.1,0.5,1,5,7,10,30,100,500], cv = 3)

    gbm_param_grid = {'learning_rate':np.array([0.2]),
                      'n_estimators':np.array([30, 100, 200, 500]),
                      'max_depth':np.array([3, 5, 13, 20]),
                      'min_child_weight':np.array([3, 10])
                      }

    Lassocv.fit(data_x_S, data_y.iloc[:,i])

    poly_Lasso_reg = Pipeline([('poly', PolynomialFeatures(degree = 2)),
                               ('std_scaler', StandardScaler()),
                               ('lasso_reg', Lasso(alpha = Lassocv.alpha_))
                              ])
    
    Lasso_model_list.append(poly_Lasso_reg)
    
    # 訓練模型，計算train, test的RMSE
    poly_Lasso_reg.fit(X_train, Y_train.iloc[:,i])
    Lasso_train_RMSE.append(np.sqrt(mean_squared_error(Y_train.iloc[:,i], poly_Lasso_reg.predict(X_train))))
    Lasso_test_RMSE.append(np.sqrt(mean_squared_error(Y_test.iloc[:,i], poly_Lasso_reg.predict(X_test))))

# 2. Ridge_model
    # Ridge超參數

    Ridgecv.fit(data_x_S, data_y.iloc[:,i])
    poly_Ridge_reg = Pipeline([('poly', PolynomialFeatures(degree = 2)),
                                                 ('std_scaler', StandardScaler()),
                                                 ('lasso_reg', Ridge(alpha = Ridgecv.alpha_))
                                                ])
    
    Ridge_model_list.append(poly_Ridge_reg)
    
    # 訓練模型，計算train, test的RMSE
    poly_Ridge_reg.fit(X_train, Y_train.iloc[:,i])  
    Ridge_train_RMSE.append(np.sqrt(mean_squared_error(Y_train.iloc[:,i], poly_Ridge_reg.predict(X_train))))
    Ridge_test_RMSE.append(np.sqrt(mean_squared_error(Y_test.iloc[:,i], poly_Ridge_reg.predict(X_test))))

# 3.XGB_model
    # XGBM超參數
    grid_search_xgbm = GridSearchCV(xgb.XGBRegressor(),
                                    param_grid = gbm_param_grid,
                                    n_jobs = -1,
                                    scoring = 'neg_root_mean_squared_error',
                                    cv = 5)
    grid_search_xgbm.fit(data_x_S, data_y)
    params = grid_search_xgbm.best_params_
    
    poly_XGB_reg = Pipeline([('poly',PolynomialFeatures(degree = 2)),
                                               ('std_scaler', StandardScaler()),
                                               ('xgb_reg', xgb.XGBRegressor(learning_rate = params['learning_rate'],
                                                                            max_depth = params['max_depth'],
                                                                            n_estimators = params['n_estimators'],
                                                                            min_child_weight = params['min_child_weight'],
                                                                            objective = 'reg:squarederror')
                                               )
                                              ])
    
    XGB_model_list.append(poly_XGB_reg)
    # 訓練模型，計算train, test的RMSE
    poly_XGB_reg.fit(X_train, Y_train.iloc[:,i])  
    XGB_train_RMSE.append(np.sqrt(mean_squared_error(Y_train.iloc[:,i], poly_XGB_reg.predict(X_train))))
    XGB_test_RMSE.append(np.sqrt(mean_squared_error(Y_test.iloc[:,i], poly_XGB_reg.predict(X_test))))
    
    
    # **各模型預測結果放入Model_result
    Model_result_train['Lasso_train_'+ data_y.columns.tolist()[i]] = poly_Lasso_reg.predict(X_train)
    Model_result_test['Lasso_test_'+ data_y.columns.tolist()[i]] = poly_Lasso_reg.predict(X_test)
    Model_result_train['Ridge_train_'+ data_y.columns.tolist()[i]] = poly_Ridge_reg.predict(X_train)
    Model_result_test['Ridge_test_'+ data_y.columns.tolist()[i]] = poly_Ridge_reg.predict(X_test)
    Model_result_train['XGB_train_'+ data_y.columns.tolist()[i]] = poly_XGB_reg.predict(X_train)
    Model_result_test['XGB_test_'+ data_y.columns.tolist()[i]] = poly_XGB_reg.predict(X_test)    
    

# list 串接成 pandas表格    
Model_RMSE = pd.DataFrame({'Lasso':Lasso_model_list,
                           'Lasso_train':Lasso_train_RMSE,
                           'Lasso_test':Lasso_test_RMSE,
                           'Ridge':Ridge_model_list,
                           'Ridge_train':Ridge_train_RMSE,
                           'Ridge_test':Ridge_test_RMSE,
                           'XGB':XGB_model_list,
                           'XGB_train':XGB_train_RMSE,
                           'XGB_test':XGB_test_RMSE
                          }, index = list(data_y.columns))

display(Model_RMSE)
display(Model_result_train)
display(Model_result_test)

,Lasso,Lasso_train,Lasso_test,Ridge,Ridge_train,Ridge_test,XGB,XGB_train,XGB_test
sensor_point5_i_value,"(PolynomialFeatures(), StandardScaler(), Lasso())",7.97,11.17,"(PolynomialFeatures(), StandardScaler(), Ridge...",5.93,11.25,"(PolynomialFeatures(), StandardScaler(), XGBRe...",3.05,10.56
sensor_point6_i_value,"(PolynomialFeatures(), StandardScaler(), Lasso...",16.59,23.30,"(PolynomialFeatures(), StandardScaler(), Ridge...",9.60,21.29,"(PolynomialFeatures(), StandardScaler(), XGBRe...",4.08,20.26
sensor_point7_i_value,"(PolynomialFeatures(), StandardScaler(), Lasso())",22.07,23.16,"(PolynomialFeatures(), StandardScaler(), Ridge...",17.97,23.80,"(PolynomialFeatures(), StandardScaler(), XGBRe...",9.73,22.88
sensor_point8_i_value,"(PolynomialFeatures(), StandardScaler(), Lasso...",17.69,18.43,"(PolynomialFeatures(), StandardScaler(), Ridge...",13.58,17.36,"(PolynomialFeatures(), StandardScaler(), XGBRe...",5.49,17.75
sensor_point9_i_value,"(PolynomialFeatures(), StandardScaler(), Lasso...",20.97,19.14,"(PolynomialFeatures(), StandardScaler(), Ridge...",14.25,17.42,"(PolynomialFeatures(), StandardScaler(), XGBRe...",7.96,17.82
sensor_point10_i_value,"(PolynomialFeatures(), StandardScaler(), Lasso...",15.48,16.00,"(PolynomialFeatures(), StandardScaler(), Ridge...",10.32,15.79,"(PolynomialFeatures(), StandardScaler(), XGBRe...",4.79,15.12


,Lasso_train_sensor_point5_i_value,Ridge_train_sensor_point5_i_value,XGB_train_sensor_point5_i_value,Lasso_train_sensor_point6_i_value,Ridge_train_sensor_point6_i_value,XGB_train_sensor_point6_i_value,Lasso_train_sensor_point7_i_value,Ridge_train_sensor_point7_i_value,XGB_train_sensor_point7_i_value,Lasso_train_sensor_point8_i_value,Ridge_train_sensor_point8_i_value,XGB_train_sensor_point8_i_value,Lasso_train_sensor_point9_i_value,Ridge_train_sensor_point9_i_value,XGB_train_sensor_point9_i_value,Lasso_train_sensor_point10_i_value,Ridge_train_sensor_point10_i_value,XGB_train_sensor_point10_i_value
0,84.90,85.27,82.56,120.24,128.02,127.36,155.03,159.58,156.84,80.39,88.22,91.21,96.97,110.20,108.20,98.54,107.22,107.27
1,82.84,83.33,84.16,95.41,82.17,76.41,110.44,111.29,121.56,78.94,79.14,88.76,91.04,92.82,96.54,89.63,79.51,87.41
2,81.78,81.41,80.95,92.89,85.33,83.27,125.50,124.78,128.79,77.90,73.84,70.77,90.47,80.50,82.97,89.96,86.08,75.31
3,80.05,80.75,76.70,91.08,85.00,85.18,122.82,125.68,123.63,77.90,72.56,74.32,90.08,74.76,75.01,90.05,91.26,108.24
4,82.78,82.14,87.71,96.13,80.37,76.20,109.16,102.34,121.36,78.88,76.67,73.86,91.25,92.99,104.76,89.59,78.39,76.70
5,79.94,79.07,85.38,74.90,70.70,71.22,99.21,86.75,87.21,98.92,107.37,108.20,97.75,82.42,68.50,98.81,97.96,97.27
6,84.28,93.90,99.90,95.21,88.19,91.84,139.11,138.92,134.28,98.16,86.39,61.94,98.91,107.02,109.48,101.66,105.34,102.26
7,76.20,77.41,80.29,65.76,54.07,60.61,78.52,68.52,57.37,54.23,50.46,48.55,80.46,61.79,49.12,80.36,68.47,72.62
8,76.36,74.38,72.06,77.17,70.13,66.90,106.25,100.02,85.67,100.90,112.64,120.05,95.52,101.55,133.39,93.58,88.32,86.53
9,67.61,63.66,58.53,99.07,110.36,93.41,119.48,126.65,99.87,75.04,78.27,92.05,92.48,105.34,83.29,96.04,103.65,126.60


,Lasso_test_sensor_point5_i_value,Ridge_test_sensor_point5_i_value,XGB_test_sensor_point5_i_value,Lasso_test_sensor_point6_i_value,Ridge_test_sensor_point6_i_value,XGB_test_sensor_point6_i_value,Lasso_test_sensor_point7_i_value,Ridge_test_sensor_point7_i_value,XGB_test_sensor_point7_i_value,Lasso_test_sensor_point8_i_value,Ridge_test_sensor_point8_i_value,XGB_test_sensor_point8_i_value,Lasso_test_sensor_point9_i_value,Ridge_test_sensor_point9_i_value,XGB_test_sensor_point9_i_value,Lasso_test_sensor_point10_i_value,Ridge_test_sensor_point10_i_value,XGB_test_sensor_point10_i_value
0,80.06,86.59,87.29,74.79,86.25,75.64,101.48,107.94,107.80,70.30,72.67,65.06,87.09,65.33,66.37,85.23,95.56,81.56
1,86.42,91.83,87.65,101.36,97.13,100.18,139.37,141.81,137.62,106.45,105.87,108.70,99.41,115.34,117.97,96.36,103.54,98.43
2,77.50,72.40,72.71,78.27,72.59,99.02,106.79,108.77,97.67,102.91,123.14,152.37,95.59,91.77,64.86,93.43,88.78,94.18
3,79.00,79.36,78.22,93.24,80.90,65.46,102.29,102.83,116.52,78.59,73.87,72.99,93.48,80.81,68.30,94.26,93.26,89.55
4,68.43,58.76,65.69,98.01,126.87,125.72,133.95,110.34,112.67,77.19,54.33,61.80,94.52,137.52,115.38,97.51,98.37,97.76
5,83.82,87.88,90.14,86.56,86.88,90.13,128.18,111.46,107.59,95.60,97.09,99.99,98.27,98.14,100.16,98.43,102.33,101.90
6,83.47,90.05,98.46,86.35,88.39,78.41,135.58,139.22,126.84,93.69,94.35,125.27,97.72,104.63,129.07,98.02,102.76,101.86
7,73.22,76.05,70.23,98.20,84.89,80.47,100.03,89.39,87.82,78.11,71.90,61.49,90.97,84.87,79.32,91.69,97.81,95.36
8,83.96,91.52,86.50,98.22,116.15,110.57,127.27,112.54,114.45,78.54,72.35,83.80,90.08,96.00,93.64,90.29,87.31,90.46
9,83.90,76.57,84.31,90.61,62.70,65.18,110.74,100.06,112.01,77.67,74.88,73.89,89.34,64.40,68.38,89.24,65.31,85.15


# 3.迴圈計算不同排列順序下，平均RMSE結果

*參考RegressionChain(Regression只能放一個模型，而且不能調整超參數)，將先預測的Y值放入X，再預測下一個Y

In [21]:
# 排列訓練模型，再計算測試、訓練集RMSE平均數。最佳排列結果取測試集及訓練集RMSE合計最小

check = 1000
order_list = []
predict_train_list = []
predict_test_list =[]
predict_train_avg = []
predict_test_avg = []

# 取得排列 0~6清單
for i in list(permutations(range(6),6)):
    train_rl = []
    test_rl = []
    X_train_PT = X_train
    X_test_PT = X_test
    
    for k in range(6):
        # 按照排列順序訓練模型
        poly_lasso_reg = Lasso_model_list[list(i)[k]]
        poly_lasso_reg.fit(X_train, Y_train.iloc[:,list(i)[k]])
        
        # lasso RMSE(train、test)
        train_rl.append(np.sqrt(mean_squared_error(Y_train.iloc[:,list(i)[k]], poly_lasso_reg.predict(X_train_PT))))
        test_rl.append(np.sqrt(mean_squared_error(Y_test.iloc[:,list(i)[k]], poly_lasso_reg.predict(X_test_PT))))
        
        # 把取得的預測值當作變數放進X(train, test)
        X_train_PT[data_y.iloc[:,[0]].columns[0]+'_predict'] = poly_lasso_reg.predict(X_train_PT)
        X_test_PT[data_y.iloc[:,[0]].columns[0]+'_predict'] = poly_lasso_reg.predict(X_test_PT)  


    order_list.append(''.join(str(list(i))))
    predict_train_avg.append(statistics.mean(train_rl))
    predict_test_avg.append(statistics.mean(test_rl))
    predict_train_list.append(str(train_rl))
    predict_test_list.append(str(test_rl))
    
    # 取 Model取 train + test後RMSE最低的結果
    if (statistics.mean(train_rl) + statistics.mean(test_rl))/2 < check:
        check = (statistics.mean(train_rl) + statistics.mean(test_rl))/2
        order_list_fn = str(list(i))


Model_train_result = pd.DataFrame({ 'Group':order_list,
                                    'Train_avg':predict_train_avg,
                                    'Test_avg':predict_test_avg,
                                    'Train_list':predict_train_list, 
                                    'Test_list':predict_test_list})

display(Model_train_result)
display(order_list_fn)

,Group,Train_avg,Test_avg,Train_list,Test_list
0,"[0, 1, 2, 3, 4, 5]",16.78,18.40,"[7.9744296848850125, 16.588704198641835, 22.26...","[11.169628927523748, 23.29730388559398, 22.709..."
1,"[0, 1, 2, 3, 5, 4]",16.82,18.44,"[7.974429684885013, 16.588704198641835, 22.264...","[11.169628927523748, 23.297303885593983, 22.70..."
2,"[0, 1, 2, 4, 3, 5]",16.75,18.36,"[7.974429675262273, 16.588704198631177, 22.264...","[11.16962890394919, 23.297303885565533, 22.709..."
3,"[0, 1, 2, 4, 5, 3]",16.70,18.31,"[7.974429613799821, 16.588704198563118, 22.264...","[11.169628753373408, 23.297303885383805, 22.70..."
4,"[0, 1, 2, 5, 3, 4]",16.81,18.42,"[7.974428910957102, 16.58870419778482, 22.2646...","[11.169627031482397, 23.297303883305712, 22.70..."
5,"[0, 1, 2, 5, 4, 3]",16.79,18.36,"[7.97442962744775, 16.588704198578235, 22.2646...","[11.169628786809241, 23.29730388542416, 22.709..."
6,"[0, 1, 3, 2, 4, 5]",16.69,18.45,"[7.974428912443557, 16.58870419778647, 17.6932...","[11.16962703512418, 23.297303883310107, 18.431..."
7,"[0, 1, 3, 2, 5, 4]",16.76,18.50,"[7.974429632194992, 16.588704198583493, 17.693...","[11.1696287984394, 23.297303885438197, 18.4311..."
8,"[0, 1, 3, 4, 2, 5]",16.78,18.55,"[7.974429684885013, 16.588704198641835, 17.693...","[11.169628927523748, 23.297303885593983, 18.43..."
9,"[0, 1, 3, 4, 5, 2]",16.75,18.48,"[7.974429685376893, 16.58870419864238, 17.6932...","[11.169628928728807, 23.297303885595436, 18.43..."


'[5, 3, 2, 1, 4, 0]'

# 4.按步驟測試模型結果

## 1.Lasso + Polymonial
* 交叉驗證取得最佳超參數
* 訓練模型
* 計算RMSE、R_square

In [ ]:
# RC_poly_Lasso_reg = RegressorChain(base_estimator = poly_Lasso_reg, order = [3,1,2,0,4,5])
# RC_poly_Lasso_reg.fit(X_train, Y_train)

In [ ]:
Lassocv =  LassoCV(alphas = [0.01,0.1,0.5,1,5,7,10,30,100,500], cv = 3, max_iter = 10000)
Lassocv.fit(data_x_S, data_y.iloc[:,0])
Lassocv.alpha_

In [ ]:
poly_lasso_reg = Pipeline([('poly',PolynomialFeatures(degree = 2)),
                          ('std_scaler', StandardScaler()),
                          ('ridge_reg', Lasso(alpha = Lassocv.alpha_))
                         ])
poly_lasso_reg.fit(X_train, Y_train.iloc[:,0])

### 訓練結果

In [ ]:
Y_predict_train = poly_lasso_reg.predict(X_train)
print('RMSE：',np.sqrt(mean_squared_error(Y_train.iloc[:,0], Y_predict_train)))
print('R_square：',r2_score(Y_train.iloc[:,0], Y_predict_train))

### 測試結果

In [ ]:
Y_predict_test = poly_lasso_reg.predict(X_test)
print('RMSE：',np.sqrt(mean_squared_error(Y_test.iloc[:,0], Y_predict_test)))
print('R_square：',r2_score(Y_test.iloc[:,0], Y_predict_test))

## 2.Ridge + Polymonial
* 交叉驗證取得最佳超參數
* 訓練模型
* 計算RMSE、R_square

In [30]:
ridgecv = RidgeCV(alphas = [0.01,0.1,0.5,1,5,7,10,30,100,500])
ridgecv.fit(data_x_S, data_y)
ridgecv.alpha_

1.0

In [59]:
poly_ridge_reg = Pipeline([('poly',PolynomialFeatures(degree = 2)),
                          ('std_scaler', StandardScaler()),
                          ('ridge_reg', Ridge(alpha = ridgecv.alpha_))
                         ])
poly_ridge_reg.fit(X_train, Y_train)

Pipeline(steps=[('poly', PolynomialFeatures()),
                ('std_scaler', StandardScaler()), ('ridge_reg', Ridge())])

### 訓練結果

In [61]:
Y_predict_train = poly_ridge_reg.predict(X_train)
print('RMSE：',np.sqrt(mean_squared_error(Y_train, Y_predict_train)))
print('R_square：',r2_score(Y_train, Y_predict_train))

RMSE： 6.0217897150510895
R_square： 0.9387506063249269


### 測試結果

In [63]:
Y_predict_test = poly_ridge_reg.predict(X_test)
print('RMSE：',np.sqrt(mean_squared_error(Y_test, Y_predict_test)))
print('R_square：',r2_score(Y_test, Y_predict_test))

RMSE： 40.83933505058546
R_square： -2.5251729174874504


## 3.XGboosting + Polymonial
* 交叉驗證取得最佳超參數
* 訓練模型
* 計算RMSE、R_square

In [ ]:
gbm_param_grid = {'learning_rate':np.array([0.2, 0.5]),
                  'n_estimators':np.array([30, 100, 200, 500]),
                  'max_depth':np.array([3, 5, 9, 13, 20]),
                  'min_child_weight':np.array([1, 3, 10])
                 }

grid_search = GridSearchCV(xgb.XGBRegressor(),
                           param_grid = gbm_param_grid,
                           n_jobs = -1,
                           scoring = 'neg_root_mean_squared_error',
                           cv = 5
                          )
# grid挑超參數時使用全部資料
grid_search.fit(data_x_S, data_y)
params = grid_search.best_params_
params

In [ ]:
poly_xgb_reg = Pipeline([('poly',PolynomialFeatures(degree = 2)),
                          ('std_scaler', StandardScaler()),
                          ('xgb_reg', xgb.XGBRegressor(learning_rate = params['learning_rate'],
                                      max_depth = params['max_depth'],
                                      n_estimators = params['n_estimators'],
                                      min_child_weight = params['min_child_weight'],
                                      objective = 'reg:squarederror'
                                      ))
                         ])
poly_xgb_reg.fit(X_train, Y_train)

### 訓練結果

In [ ]:
Y_predict_train = poly_xgb_reg.predict(X_train)
print('RMSE：',np.sqrt(mean_squared_error(Y_train, Y_predict_train)))
print('R_square：',r2_score(Y_train, Y_predict_train))

### 測試結果

In [ ]:
Y_predict_test = poly_xgb_reg.predict(X_test)
print('RMSE：',np.sqrt(mean_squared_error(Y_test, Y_predict_test)))
print('R_square：',r2_score(Y_test, Y_predict_test))

### 並非所有回歸算法都支持多輸出回歸(例如：支持向量機SVR)，可以透過MultiOutputRegressor或RegressorChain來達到多輸出回歸的目標，但兩者功能上有些差異：
*1.MultiOutputRegressor：假設輸出彼此獨立，為每個輸出創建提供模型。

*2.RegressorChain：序列中的第一個模型使用輸入並預測一個輸出。第二模型使用第一模型的輸入和輸出進行預測；第三個模型使用前兩個模型的輸入和輸出進行預測，依此類推。(多輸出之間具有順序性，可透過"order"參數指定。例如，order = [0,1]將首先預測第0個輸出，再預測第一個輸出。

## 4.XGboosting + Polymonial(RegressorChain)

In [ ]:
check = 1000
order_list = []
predict_train_list = []
predict_test_list =[]

for i in list(permutations(range(6),6)):
    RC_poly_xgb_reg = RegressorChain(base_estimator = poly_xgb_reg, order = list(i))
    RC_poly_xgb_reg.fit(X_train, Y_train)

    order_list.append(''.join(str(i)))
    predict_train_list.append(np.sqrt(mean_squared_error(Y_train, RC_poly_xgb_reg.predict(X_train))))
    predict_test_list.append(np.sqrt(mean_squared_error(Y_test, RC_poly_xgb_reg.predict(X_test))))

     # Model取 train + test後RMSE最低的結果
    if np.sqrt(mean_squared_error(Y_train, RC_poly_xgb_reg.predict(X_train))) + np.sqrt(mean_squared_error(Y_test, RC_poly_xgb_reg.predict(X_test))) < check:
        check = np.sqrt(mean_squared_error(Y_train, RC_poly_xgb_reg.predict(X_train))) + np.sqrt(mean_squared_error(Y_test, RC_poly_xgb_reg.predict(X_test)))
        RC_poly_xgb_reg_fn = RC_poly_xgb_reg

Model_train_result = pd.DataFrame({ 'Group':order_list, 'Train':predict_train_list, 'Test':predict_test_list})
display(Model_train_result)
display(RC_poly_xgb_reg_fn)

### 訓練結果(RegressorChain)

In [ ]:
RC_Y_predict_train = RC_poly_xgb_reg.predict(X_train)
print('RMSE：',np.sqrt(mean_squared_error(Y_train, RC_Y_predict_train)))
print('R_square：',r2_score(Y_train, RC_Y_predict_train))

### 測試結果(RegressorChain)

In [ ]:
RC_Y_predict_test = RC_poly_xgb_reg.predict(X_test)
print('RMSE：',np.sqrt(mean_squared_error(Y_test, RC_Y_predict_test)))
print('R_square：',r2_score(Y_test, RC_Y_predict_test))

### 比對預測值跟實際值差異

In [ ]:
Y_test_predict_diff = Y_test

for i in range(6):
    Y_test_predict_diff[Y_test.columns.tolist()[i] + '_predict'] = RC_Y_predict_test.T[i]
    Y_test_predict_diff[Y_test.columns.tolist()[i] + '_diff'] = Y_test[Y_test.columns.tolist()[i]] - RC_Y_predict_test.T[i]

display(Y_test_predict_diff)

In [ ]:
fig, axes = plt.subplots(6, 2, figsize=(12, 18))

for i in range(6):
    sns.regplot(x = Y_test.columns.tolist()[i], 
                y = Y_test.columns.tolist()[i] + '_predict', 
                data = Y_test_predict_diff, 
                ax=axes[i, 0])

    sns.scatterplot(x=Y_test_predict_diff.index,
                    y=Y_test.columns.tolist()[i] + '_diff',
                    data = Y_test_predict_diff, 
                    ax=axes[i, 1])
    
    
plt.tight_layout()

## 4.KNeighborsRegressor

In [ ]:
gbm_param_grid = {'n_neighbors':np.array([2, 3, 5, 7]),
                  'Leaf_size':np.array([20, 30, 50]),
                 }

grid_search = GridSearchCV(KNeighborsRegressor(),
                           param_grid = gbm_param_grid,
                           n_jobs = -1,
                           scoring = 'neg_root_mean_squared_error',
                           cv = 5
                          )
# grid挑超參數時使用全部資料
grid_search.fit(data_x_S, data_y)

params = grid_search.best_params_
params

In [ ]:
poly_KN_reg = Pipeline([('poly',PolynomialFeatures(degree = 2)),
                        ('std_scaler', StandardScaler()),
                        ('KNR', KNeighborsRegressor(n_neighbors = params['n_neighbors'],
                                                    Leaf_size = params['Leaf_size']
                                                    ))
                        ])
poly_KN_reg.fit(X_train, Y_train)

In [ ]:
Y_predict_train = poly_KN_reg.predict(X_train)
print('RMSE：',np.sqrt(mean_squared_error(Y_train, Y_predict_train)))
print('R_square：',r2_score(Y_train, Y_predict_train))

In [ ]:
Y_predict_test = poly_KN_reg.predict(X_test)
print('RMSE：',np.sqrt(mean_squared_error(Y_test, Y_predict_test)))
print('R_square：',r2_score(Y_test, Y_predict_test))

### Test：檢查MultiOutputRegressor跑出來的結果跟沒有用MultiOutputRegressor的結果是否有差異

In [ ]:
wrapper = MultiOutputRegressor(poly_xgb_reg)
wrapper.fit(X_train, Y_train)
Y_predict_MO = wrapper.predict(X_train)
Y_predict_MO - Y_predict

### MixedRandomForest
*n_estimators(int)：森林中的樹木數量。可選的。默認值：10。

*max_features(int | float | str)：尋找最佳分割時要考慮的特徵數量。可選的。默認值：'sqrt'。
如果是 int，則在每次拆分時考慮 max_features 特徵。
如果是浮點數，那麼 max_features 是一個分數，並且在每次拆分時都會考慮 int(max_features * n_features) 個特徵。
如果“sqrt”，則 max_features=sqrt(n_features)（與“auto”相同）。
如果“log2”，則 max_features=log2(n_features)。
如果沒有，那麼 max_features=n_features。
注意：在找到至少一個節點樣本的有效分區之前，對拆分的搜索不會停止，即使它需要有效地檢查超過 max_features 個特徵。

*min_samples_leaf(int)：葉節點所需的最小樣本數。可選的。默認值：5。

注意：任何深度的分割點只有在左右分支中至少留下 min_samples_leaf 訓練樣本時才會被考慮。這可能具有平滑模型的效果，尤其是在回歸中。

*choose_split(str)：用於找到最佳分割的方法。可選的。默認值：“平均值”。

默認情況下，將使用平均信息增益。

可能的值：
'mean'：使用平均信息增益。
'max'：使用最大信息增益。
'random'：隨機選擇其中一個預測任務，並選擇其個體信息增益作為分割的信息增益。
classification_targets(int[])：屬於分類任務的特徵。可選的。默認值：無。

如果未指定分類目標，則隨機森林會將所有變量視為回歸變量。

In [ ]:
# check = 1000
# order_list = []
# predict_train_list = []
# predict_test_list =[]
# for a in range(6):
#     for b in range(6):
#         for c in range(6):
#             for d in range(6):
#                 for e in range(6):
#                     for f in range(6):
#                         if a not in set([b,c,d,e,f]) and \
#                            b not in set([c,d,e,f]) and \
#                            c not in set([d,e,f]) and \
#                            d not in set([e,f]) and \
#                            e not in set([f]):
#                             RC_poly_xgb_reg = RegressorChain(base_estimator = poly_xgb_reg, order = [a,b,c,d,e,f])
#                             RC_poly_xgb_reg.fit(X_train, Y_train)
                            
                            
#                             order_list.append(str(a)+str(b)+str(c)+str(d)+str(e)+str(f))
#                             predict_train_list.append(np.sqrt(mean_squared_error(Y_train, RC_poly_xgb_reg.predict(X_train))))
#                             predict_test_list.append(np.sqrt(mean_squared_error(Y_test, RC_poly_xgb_reg.predict(X_test))))
                            
#                             # Model取 train + test後RMSE最低的結果
#                             if np.sqrt(mean_squared_error(Y_train, RC_poly_xgb_reg.predict(X_train))) \
#                                + np.sqrt(mean_squared_error(Y_test, RC_poly_xgb_reg.predict(X_test))) < check:
#                                 check = np.sqrt(mean_squared_error(Y_train, RC_poly_xgb_reg.predict(X_train))) \
#                                 + np.sqrt(mean_squared_error(Y_test, RC_poly_xgb_reg.predict(X_test)))
#                                 RC_poly_xgb_reg_fn = RC_poly_xgb_reg

# Model_train_result = pd.DataFrame({ 'Group':order_list, 'Train':predict_train_list, 'Test':predict_test_list})
# display(Model_train_result)
# display(RC_poly_xgb_reg_fn)

In [ ]:
# check = 1000
# order_list = []
# predict_train_list = []
# predict_test_list =[]

# for i in list(permutations(range(6),6)):
#     RC_poly_lasso_reg = RegressorChain(base_estimator = [Lasso_model_list[list(i)[0]],
#                                                        Lasso_model_list[list(i)[1]],
#                                                        Lasso_model_list[list(i)[2]],
#                                                        Lasso_model_list[list(i)[3]],
#                                                        Lasso_model_list[list(i)[4]],
#                                                        Lasso_model_list[list(i)[5]]
#                                                        ], order = list(i))
#     RC_poly_lasso_reg.fit(X_train, Y_train)

#     order_list.append(''.join(str(i)))
#     predict_train_list.append(np.sqrt(mean_squared_error(Y_train, RC_poly_lasso_reg.predict(X_train))))
#     predict_test_list.append(np.sqrt(mean_squared_error(Y_test, RC_poly_lasso_reg.predict(X_test))))

#      # Model取 train + test後RMSE最低的結果
#     if np.sqrt(mean_squared_error(Y_train, RC_poly_lasso_reg.predict(X_train))) + np.sqrt(mean_squared_error(Y_test, RC_poly_lasso_reg.predict(X_test))) < check:
#         check = np.sqrt(mean_squared_error(Y_train, RC_poly_lasso_reg.predict(X_train))) + np.sqrt(mean_squared_error(Y_test, RC_poly_lasso_reg.predict(X_test)))
#         RC_poly_lasso_reg_fn = RC_poly_lasso_reg

# Model_train_result = pd.DataFrame({ 'Group':order_list, 'Train':predict_train_list, 'Test':predict_test_list})
# display(Model_train_result)
# display(RC_poly_lasso_reg_fn)

In [ ]:
# Y_predict_test = RC_poly_Lasso_reg.predict(X_test)
# print('RMSE：',np.sqrt(mean_squared_error(Y_test, Y_predict_test)))
# print('R_square：',r2_score(Y_test, Y_predict_test))

### K-fold驗算是否過擬
*n_spilits 折數; n_repeats 交叉驗證器需要重複的次數

In [ ]:
cv_scores = []
#cv = RepeatedKFold(n_splits = 5, n_repeats = 3, random_state = 1)
cv = 5
scores = cross_val_score(RC_poly_xgb_reg, data_x, data_y, cv = cv, scoring = 'neg_root_mean_squared_error')
cv_scores.append(-scores.mean())
cv_scores